In [1]:
import json
import pandas as pd
import warnings
import seaborn as sns
from pyprojroot import here

methods = ["TimeGAN", "Time-Transformer", "TTS-GAN", "TransFusion"]
datasets = ["D2", "D3", "D4", "D5", "D6", "D7"]

rows = []

for method in methods:
    for dataset in datasets:
        file_candidates = list(here('result').glob(f'numeric_{method}_{dataset}_*.json'))
        if len(file_candidates) != 1:
            warnings.warn(f"Ignoring {method} {dataset}: Expected one result file for {method} {dataset}, instead matched {file_candidates}.")
            continue
        file_path = file_candidates[0]
        
        with open(file_path) as f:
            data = json.load(f)
        
        rows.append(((method, dataset), data))

df = pd.DataFrame.from_dict(dict(rows), orient="index")
df.index = pd.MultiIndex.from_tuples(df.index, names=["Method", "Dataset"])

df

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring TTS-GAN D2: Expected one result file for TTS-GAN D2, instead matched [].
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring TTS-GAN D3: Expected one result file for TTS-GAN D3, instead matched [].
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring TTS-GAN D4: Expected one result file for TTS-GAN D4, instead matched [].
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring TTS-GAN D5: Expected one result file for TTS-GAN D5, instead matched [].
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring TTS-GAN D6: Expected one result file for TTS-GAN D6, instead matched [].
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring TTS-GAN D7: Expected one result file for TTS-GAN D7, instead matched [].
  
/usr/local/lib/python3

DS        PS     C-FID       MDD       ACD  \
Method           Dataset                                                     
TimeGAN          D2       0.112879  0.038129  0.019055  0.434670  0.171891   
                 D3       0.362715  0.041257  0.050287  0.870274  0.143190   
                 D4       0.116530  0.133363  0.105101  0.766039  0.127232   
                 D5       0.472527  0.285059  0.154532  0.382266  0.451997   
                 D6       0.500000  0.309733  0.635030  0.741083  2.335000   
                 D7       0.011219  0.000890  0.000891  2.217216  1.864680   
Time-Transformer D2       0.315152  0.085012  0.258219  0.627615  0.247822   
                 D3       0.312012  0.072252  0.177665  0.747522  0.486708   
                 D4       0.124348  0.066113  0.045602  0.517450  0.308017   
                 D5       0.499859  0.326896  0.147380  0.389958  0.266162   
                 D6       0.500000  0.251982  0.456342  0.614462  0.701291   
                 D7       0.480366  0.002254  0.001517  9.074205  1.281966   
TransFusion      D2       0.150000  0.037576  0.090272  0.793641  0.033649   
                 D3       0.000780  0.037371  0.004399  0.404956  0.111957   
                 D5       0.118625  0.251417  0.009846  0.227218  0.062980   

                                 SD           KD        ED        DTW  
Method           Dataset                                               
TimeGAN          D2        0.482856     2.351818  1.189109   3.068079  
                 D3        0.262877     2.341035  2.646002   6.679519  
                 D4        0.194281     0.496604  2.857787   9.064095  
                 D5        0.263064     1.007865  1.057500   6.458857  
                 D6        0.547243     0.918373  2.610278  14.721798  
                 D7       73.477606  8942.861949  0.050241   0.255532  
Time-Transformer D2        0.592418     2.517755  1.307718   3.375031  
                 D3        0.522732     2.953833  2.561868   6.510620  
                 D4        0.261743     0.376560  2.692303   8.484202  
                 D5        0.204340     0.836956  0.950273   5.980854  
                 D6        0.417475     1.213679  2.451943  14.490414  
                 D7       73.553105  8942.827292  0.106958   0.435572  
TransFusion      D2        0.533438     2.629961  1.072507   2.773607  
                 D3        0.175680     0.824398  2.779363   7.092978  
                 D5        0.108366     0.426964  1.012462   6.289462

In [2]:

cm = sns.diverging_palette(h_neg=130, h_pos=0, as_cmap=True)
#cm = sns.light_palette("green", as_cmap=True, reverse=True)

df.swaplevel("Method", "Dataset")\
    .sort_index(level=["Dataset", "Method"])\
    .style\
        .background_gradient(cmap=cm, vmin=0, vmax=0.5, subset=["DS", "PS"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["C-FID"])\
        .background_gradient(cmap=cm, vmin=0, vmax=2, subset=["MDD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["ACD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["SD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["KD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["ED"])\
        .background_gradient(cmap=cm, vmin=0, vmax=10, subset=["DTW"])

In [3]:
df.swaplevel("Method", "Dataset")\
    .sort_index(level=["Dataset", "Method"])\
    .style\
        .background_gradient(cmap=cm, vmin=0, vmax=df["DS"].quantile(.90).max(), subset=["DS", "PS"])\
        .background_gradient(cmap=cm, vmin=0, vmax=df["C-FID"].quantile(.90).max(), subset=["C-FID"])\
        .background_gradient(cmap=cm, vmin=0, vmax=df["MDD"].quantile(.90).max(), subset=["MDD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=df["ACD"].quantile(.90).max(), subset=["ACD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=df["SD"].quantile(.90).max(), subset=["SD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=df["KD"].quantile(.90).max(), subset=["KD"])\
        .background_gradient(cmap=cm, vmin=0, vmax=df["ED"].quantile(.90).max(), subset=["ED"])\
        .background_gradient(cmap=cm, vmin=0, vmax=df["DTW"].quantile(.90).max(), subset=["DTW"])

In [4]:

df.style\
    .background_gradient(cmap=cm, vmin=0, vmax=0.5, subset=["DS", "PS"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["C-FID"])\
    .background_gradient(cmap=cm, vmin=0, vmax=2, subset=["MDD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["ACD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["SD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["KD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["ED"])\
    .background_gradient(cmap=cm, vmin=0, vmax=15, subset=["DTW"])

In [5]:
df.style\
    .background_gradient(cmap=cm, vmin=0, vmax=df["DS"].quantile(.90).max(), subset=["DS", "PS"])\
    .background_gradient(cmap=cm, vmin=0, vmax=df["C-FID"].quantile(.90).max(), subset=["C-FID"])\
    .background_gradient(cmap=cm, vmin=0, vmax=df["MDD"].quantile(.90).max(), subset=["MDD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=df["ACD"].quantile(.90).max(), subset=["ACD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=df["SD"].quantile(.90).max(), subset=["SD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=df["KD"].quantile(.90).max(), subset=["KD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=df["ED"].quantile(.90).max(), subset=["ED"])\
    .background_gradient(cmap=cm, vmin=0, vmax=df["DTW"].quantile(.90).max(), subset=["DTW"])

In [6]:
import plotly.express as px

px.line_polar(df.xs("D2", level="Dataset"), r='', theta='')

ValueError: Value of 'r' is not the name of a column in 'data_frame'. Expected one of ['DS', 'PS', 'C-FID', 'MDD', 'ACD', 'SD', 'KD', 'ED', 'DTW'] but received: 